In [1]:
#Importing libraries
import bempp.api
import numpy as np 
import dolfin
from readoff import *
from readpqr import *
import trimesh
import time
start = time.time()

In [2]:
#Main Data to modify.
#Physical parameters.
em = 4.           #[-] Interior electrical permittivity of the solute.
es = 80.          #[-] Exterior electrical permittivity of the solvent.
ks = 0.125        #[1/A] Inverse of the Debey-Huckel length of the fluid in the solvent.

#Choice of solute.
MeshV = 'Mallas_V/outputTetMeshSphere58R4T11.xml'  #Location of the volumetric mesh of the solute for which the energy is to be calculated. In "xml" format.
PQR  =  'PQR/Sphere5Q3.pqr'              #Location of the position and charges of the solute for which the energy is to be calculated. In "pqr" format.
FileA = 'Lista_A/Alfa_Sphere58R4T11.txt' #Location of the solute alpha values file to work with variable permittivities. In "txt" format. If not necessary, place ''.

#Assembly of border operators.
#fmm: For molecules with a greater number of vertices.
#default_nonlocal: For molecules with a small number of vertices.    
Assemble = 'default_nonlocal' 

#Important parameters of the GMRES.
Tol =1e-6    #GMRES tolerance.
Res =100     #Restart of the GMRES in each iteration.

#Choice of variable permittivity method in the intermediate domain.
# 'C' if it is Constant.
# 'VL' if it is for Linear variable.
# 'VTH' if it is by Hyperbolic Tangent variable.
# 'VR' if it is by variable by Radius of the solute atoms.
Va = 'VTH' 

#For the case of Hyperbolic Tangent.
kp = 'N'                     #[-] Transition slope of the Hyperbolic Tangent from 0 to infinity. 'N'for kp tending to infinity.
#For the constant and radius variable case.
ei = 10.         #[-] Exterior electrical permittivity of the intermediate domain.
ki = 0           #[1/A] Inverse of the Debey-Huckel length of the fluid in the intermediate domain.

#Secondary parameters when working with the fmm assembly.
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5  
bempp.api.GLOBAL_PARAMETERS.fmm.ncrit = 400          
bempp.api.GLOBAL_PARAMETERS.quadrature.regular = 4  

In [3]:
#Data on the position in space, charge and radii of the atoms of the molecule.
PC,Q,R = readpqr(PQR)   

In [4]:
#Import the volumetric mesh of the solute.
from dolfin import Mesh
mesh = Mesh(MeshV)  #Creation of the volumetric mesh.

In [5]:
#Generate global functional spaces of the potential in Fem and its derivative in Bem.
from bempp.api.external import fenics
fenics_space = dolfin.FunctionSpace(mesh, "CG", 1)  #Electrostatic potential at the interface and domain of the solute.
trace_space, trace_matrix = \
    fenics.fenics_to_bempp_trace_data(fenics_space) #Global trace space to work in BEM and FEM simultaneously.

In [6]:
#Process to separate the trace_space for the case of the inner mesh and the outer mesh individually.
#Code to identify vertices and faces of the inner and outer mesh.
faces0 = trace_space.grid.elements
vertices0 = trace_space.grid.vertices
meshSP = trimesh.Trimesh(vertices = vertices0.transpose(), faces= faces0.transpose())
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split))
V1 = len(mesh_split[0].vertices)
F1 = len(mesh_split[0].faces)

#Obtaining the inner surface mesh.
faces1 = faces0.transpose()[:F1]
vertices1 = vertices0.transpose()[:V1]
grid1 = bempp.api.grid.grid.Grid(vertices1.transpose(), faces1.transpose())
bempp_space1 = bempp.api.function_space(grid1, "P", 1) # Derived from the electrostatic potential at the inner interface.
trace_space1 = bempp.api.function_space(grid1, "P", 1) # Trace space to work at inner BEM and FEM simultaneously.

#Obtaining the outer surface mesh.
faces2 = faces0.transpose()[F1:]
vertices2 = vertices0.transpose()[V1:]
grid2 = bempp.api.grid.grid.Grid(vertices2.transpose(), (faces2-len(vertices1)).transpose())
bempp_space2 = bempp.api.function_space(grid2, "P", 1) # Derived from the electrostatic potential at the upper interface.
trace_space2 = bempp.api.function_space(grid2, "P", 1) # Trace space to work at outer BEM and FEM simultaneously.

#Element visualization.
print("FEM dofs: {0}".format(mesh.num_vertices()))
print("BEM1 dofs: {0}".format(bempp_space1.global_dof_count))
print("BEM2 dofs: {0}".format(bempp_space2.global_dof_count))
print("Tra1 dofs: {0}".format(trace_space1.global_dof_count))
print("Tra2 dofs: {0}".format(trace_space2.global_dof_count))
print("TraL dofs: {0}".format(trace_space.global_dof_count))

Found 2 meshes
FEM dofs: 50529
BEM1 dofs: 4591
BEM2 dofs: 11924
Tra1 dofs: 4591
Tra2 dofs: 11924
TraL dofs: 16515


In [7]:
#Process to separate the trace_matrix for the case of the lower mesh and the upper mesh individually.
Nodos = np.zeros(trace_space.global_dof_count)
Lista_Vertices = []

#Procedure to locate the vertices of the lower trace in the global trace.
for i in range(len(trace_space1.grid.vertices.T)):
    valores = np.linalg.norm(trace_space1.grid.vertices[:, i] - trace_space.grid.vertices.T,axis= 1)
    index = np.argmin(valores)
    Lista_Vertices.append(index)
    
Nodos[Lista_Vertices] = 1
trace_matrix1 = trace_matrix[Nodos.astype(bool)]
trace_matrix2 = trace_matrix[np.logical_not(Nodos)]

In [8]:
#Generate the boundary operators.
#Identity operators.
I1 = bempp.api.operators.boundary.sparse.identity(trace_space1, bempp_space1, bempp_space1) # 1
I2 = bempp.api.operators.boundary.sparse.identity(trace_space2, bempp_space2, bempp_space2) # 1
mass1 = bempp.api.operators.boundary.sparse.identity(bempp_space1, trace_space1, trace_space1) # 1
mass2 = bempp.api.operators.boundary.sparse.identity(bempp_space2, trace_space2, trace_space2) # 1

#Domain of solute Ωm in BEM.
K1 = bempp.api.operators.boundary.laplace.double_layer(trace_space1, bempp_space1, bempp_space1, assembler=Assemble) #K
V1 = bempp.api.operators.boundary.laplace.single_layer(bempp_space1, bempp_space1, bempp_space1, assembler=Assemble) #V
Z1 = bempp.api.ZeroBoundaryOperator(bempp_space2, bempp_space1, bempp_space1) #0

#Domain of solvent Ωs in BEM.
Z2 = bempp.api.ZeroBoundaryOperator(bempp_space1, bempp_space2, bempp_space2) #0
if ks==0:
    K2 = bempp.api.operators.boundary.laplace.double_layer(trace_space2, bempp_space2, bempp_space2, assembler=Assemble) #K
    V2 = bempp.api.operators.boundary.laplace.single_layer(bempp_space2, bempp_space2, bempp_space2, assembler=Assemble) #V  
else:
    K2 = bempp.api.operators.boundary.modified_helmholtz.double_layer(trace_space2, bempp_space2, bempp_space2, ks, assembler=Assemble) #K
    V2 = bempp.api.operators.boundary.modified_helmholtz.single_layer(bempp_space2, bempp_space2, bempp_space2, ks, assembler=Assemble) #V

In [9]:
#Define Dolfin's functional space.
u = dolfin.TrialFunction(fenics_space)
v = dolfin.TestFunction(fenics_space)

In [10]:
#Definition of the variable functions of the intermediate domain.
def Lista_Alfa(File): #Function to obtain the alpha parameter of the text and save it in a list.
    L_Alfa = []
    with open(File,"r") as f:  
        lines = f.readlines()
    for line in lines:
        line = line.split()
        L_Alfa.append(float(line[0]))
    return L_Alfa

#Case ei by variable by Hyperbolic Tangent. 
class Fun_ei_Tangente_Hiperbolica(dolfin.UserExpression):  
    def __init__(self,kp,L_Alfa,**kwargs):
        super().__init__(**kwargs)
    def eval_cell(self, v, x, ufc_cell):
        alfa = L_Alfa[ufc_cell.index]
        if kp=='N':  #Optional for the Hyperbolic Tangent function with kp equal to infinity.
            if alfa<0.5:
                v[0] = 1
            else:
                v[0] = 0        
        else:
            v[0] = 0.5-0.5*np.tanh(kp*(alfa-0.5)) #Hyperbolic tangent function for kp other than infinity.         
    def value_shape(self):
        return ()  

#Case ei for Linear variable.
class Fun_ei_Lineal(dolfin.UserExpression):  
    def __init__(self,L_Alfa,**kwargs):
        super().__init__(**kwargs)
    def eval_cell(self, v, x, ufc_cell):
        v[0] = L_Alfa[ufc_cell.index]  #The calculated alpha corresponds to the same linear interpolation between the two surface meshes.
    def value_shape(self):
        return ()  

#Case ei by variable by Radius.
class Fun_ei_Radio(dolfin.UserExpression):  
    def __init__(self,PC,R,em,ei,**kwargs):
        super().__init__(**kwargs)
        self.PC = PC
        self.R = R
        self.em = em
        self.ei = ei
    def eval(self, v, x):
        PC = self.PC
        R = self.R
        em = self.em
        ei = self.ei
        d0 = np.linalg.norm( x - PC, axis=1) #Minimum distance to the atoms of the molecule.
        i0 = np.argsort(d0) 
        ro = (R[i0[0]]/d0[i0[0]])/(1/d0[i0[0]])+1.4 #Effective radius.
        ef = (ei-1+55/26)-(5/13)*ro #Effective epsilon per vertex.
        if(ef>em):
            v[0] = ef #Radius distribution per tetrahedron.
        else:
            v[0] = em #In case ef is less than em, it is changed to em due to the physical sense of permittivity.
    def value_shape(self):
        return () 

if Va=='C':  #Constant Case.
    EI = ei
    K  = EI*ki**2
elif Va=='VL':  #Linear variable case.
    L_Alfa = Lista_Alfa(FileA) 
    S = Fun_ei_Lineal(L_Alfa,degree=0)
    EI = em+(es-em)*S 
    K  = S*(es*ks**2)
elif Va=='VTH': #Case of variable by Hyperbolic Tangent.
    L_Alfa = Lista_Alfa(FileA) 
    S = Fun_ei_Tangente_Hiperbolica(kp,L_Alfa,degree=0)
    EI = es+(em-es)*S
    K = (1-S)*(es*ks**2)
elif Va=='VR': #Variable case by Radius of the solute atoms.
    EI =Fun_ei_Radio(PC,R,em,ei,degree=0)
    K = EI*ki**2

In [11]:
#Construction left 3x3 matrix.
from bempp.api.external.fenics import FenicsOperator
from scipy.sparse.linalg import LinearOperator
from bempp.api.assembly.blocked_operator import BlockedDiscreteOperator
blocks = [[None,None,None],[None,None,None],[None,None,None]]

trace_op1 = LinearOperator(trace_matrix1.shape, lambda x:trace_matrix1*x)
trace_op2 = LinearOperator(trace_matrix2.shape, lambda x:trace_matrix2*x)
A = FenicsOperator((EI*dolfin.inner(dolfin.nabla_grad(u),dolfin.nabla_grad(v))+ K*u*v) * dolfin.dx)

#Position of the 3x3 matrix.
blocks[0][0] = V1.weak_form()                     # -V
blocks[0][1] = (0.5*I1-K1).weak_form()*trace_op1  # 0.5+K
blocks[0][2] = Z1.weak_form()                     # 0
blocks[1][0] = -trace_matrix1.T*em*mass1.weak_form().A   # -Mg1
blocks[1][1] =  A.weak_form()                      # A
blocks[1][2] = -trace_matrix2.T*es*mass2.weak_form().A   # -Mg2
blocks[2][0] = Z2.weak_form()                     # 0
blocks[2][1] = (0.5*I2-K2).weak_form()*trace_op2  # 0.5-K
blocks[2][2] = V2.weak_form()                     # V
blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks))  

In [12]:
#Creation of Coulomb potential function.
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em
    result[:] = (1 / (4.*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
Uca = bempp.api.GridFunction(bempp_space1, fun=U_c)

#Construction of the right vector.
# Rhs in Ωm in BEM.
rhs_bem1 = (Uca).projections(bempp_space1)
# Rhs in Ωi in FEM.
rhs_fem =  np.zeros(mesh.num_vertices()) 
# Rhs in Ωs in BEM.
rhs_bem2 = np.zeros(bempp_space2.global_dof_count) 
# The combination of rhs.
rhs = np.concatenate([rhs_bem1, rhs_fem, rhs_bem2])

In [13]:
#Creation of the Mass Matrix preconditioner for BEM/FEM/BEM.
from bempp.api.assembly.discrete_boundary_operator import InverseSparseDiscreteBoundaryOperator
from scipy.sparse.linalg import LinearOperator
from scipy.sparse import diags

P2 = diags(1./(blocked[1,1].A).diagonal())
P1 = InverseSparseDiscreteBoundaryOperator(
    bempp.api.operators.boundary.sparse.identity(
        bempp_space1, bempp_space1, bempp_space1).weak_form())
P3 = InverseSparseDiscreteBoundaryOperator(
    bempp.api.operators.boundary.sparse.identity(
        bempp_space2, bempp_space2, bempp_space2).weak_form())

def apply_prec(x):
    """Apply the block diagonal preconditioner"""
    m1 = P1.shape[0]
    m2 = P2.shape[0]
    m3 = P3.shape[0]
    n1 = P1.shape[1]
    n2 = P2.shape[1]
    n3 = P3.shape[1]
 
    res1 = P1.dot(x[:n1])
    res2 = P2.dot(x[n1: n1+n2])
    res3 = P3.dot(x[n1+n2:])
    return np.concatenate([res1, res2, res3])

p_shape = (P1.shape[0] + P2.shape[0] + P3.shape[0], P1.shape[1] + P2.shape[1] + P3.shape[1])
P = LinearOperator(p_shape, apply_prec, dtype=np.dtype('complex128'))

/home/mauricioguerrero/Software/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [14]:
#The solution of the matrix equation Ax=B is solved.
#Iteration counter.
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 100) == (it_count // 100):
        print(it_count,x)

# Solution by GMRES.
from scipy.sparse.linalg import gmres
start1 = time.time()
soln, info = gmres(blocked, rhs, M=P, callback=count_iterations,tol=Tol, restart=Res)  
end1 = time.time() 

# Time to solve the equation.
curr_time1 = (end1 - start1)
print("Number of GMRES iterations: {0}".format(it_count))
print("Total time in GMRES: {:5.2f} [s]".format(curr_time1))   

100 2.0328161408022426e-06
Number of GMRES iterations: 164
Total time in GMRES: 128.08 [s]


In [15]:
#Calculate the entire global domain of the potential from the solution of the edges of both interfaces.
soln_bem1 = soln[:bempp_space1.global_dof_count] 
soln_fem  = soln[bempp_space1.global_dof_count : bempp_space1.global_dof_count + mesh.num_vertices()]
soln_bem2 = soln[bempp_space1.global_dof_count + mesh.num_vertices():]
# Calculate the solution of the real potential in the FEM domain in the intermediate region.
u = dolfin.Function(fenics_space)
u.vector()[:] = np.ascontiguousarray(np.real(soln_fem)) 
# Solution for Dirichlet data in the inner interface.
dirichlet_data1 = trace_matrix1 * soln_fem
dirichlet_fun1 = bempp.api.GridFunction(trace_space1, coefficients=dirichlet_data1)
# Solution for Neumann data in the inner interface.
neumann_fun1 = bempp.api.GridFunction(bempp_space1, coefficients=soln_bem1)
# Solution for Dirichlet data in the outer interface.
dirichlet_data2 = trace_matrix2 * soln_fem
dirichlet_fun2 = bempp.api.GridFunction(trace_space2, coefficients=dirichlet_data2)
# Solution for Neumann data in the outer interface.
neumann_fun2 = bempp.api.GridFunction(bempp_space2, coefficients=soln_bem2)

In [16]:
#Calculation of the potential in the position of the atoms of the molecule.
VF1 = bempp.api.operators.potential.laplace.single_layer(bempp_space1, np.transpose(PC)) 
KF1 = bempp.api.operators.potential.laplace.double_layer(trace_space1, np.transpose(PC))
uF = -VF1*neumann_fun1 + KF1*dirichlet_fun1 

#Result of the total solvation energy.
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Solvation Energy: {:7.6f} [kCal/mol]'.format(E_Solv) )

#Total time.
end = time.time()
curr_time = (end - start)
print("Total time: {:5.2f} [s]".format(curr_time))

Solvation Energy: -55.954595 [kCal/mol]
Total time: 398.23 [s]
